## 패키지 Import, 환경변수

In [4]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv() #환경변수

MONGO_URI = os.getenv("MONGODB_URI")
DB_NAME = os.getenv("MONGODB_DB_NAME")
COLLECTION_NAME = 'si'

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.0)

# MongoDB

In [7]:
from typing import Dict, List

#data fatch 함수
def fetch_data_from_mongodb(collection_name: str, query: Dict = None, limit: int = None) -> List[Dict]:
    #query: Dict = None : 딕셔너리 형태로 query 가져오고 기본 None
    #> List[Dict]: 함수의 반환 값에 대한 타입
 

    client = MongoClient(MONGO_URI) #데이터 서버 연결
    db = client[DB_NAME] #데이터 베이스 선택
    collection = db[COLLECTION_NAME] #컬렉션 선택
    
    # Prepare the find operation
    find_operation = collection.find(query) if query else collection.find()
    
    # Fetch and return the data
    data = list(find_operation)
    
    # Close the connection
    client.close()
    
    return data

# Langgraph

### Langgraph funtions

In [9]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from bson import json_util

def check_missing(data):
    prompt = PromptTemplate(
        template="Are there any missing data with the following collection: {data}?",
        input_variables=["data"]
    )
    llm = ChatOpenAI()  # You might want to customize this
    chain = prompt | llm | StrOutputParser()
    try:
        response = chain.invoke({"data": json_util.dumps(data)})
        #mongodb에서 가져온 데이터를 Json으로 parse
        return response
    except Exception as e:
        raise RuntimeError(f"Error during checking missing data: {e}")

In [10]:
def summarize_data(data, missing_info):
    prompt = PromptTemplate(
        template="Summarize the main information and missing data from this collection:\n\nData: {data}\n\nMissing Information: {missing_info}",
        input_variables=["data", "missing_info"]
    )
    chain = prompt | llm | StrOutputParser()
    try:
        response = chain.invoke({"data": json_util.dumps(data), "missing_info": missing_info})
        return response
    except Exception as e:
        raise RuntimeError(f"Error during summarizing data: {e}")

In [15]:
from typing import Optional, Annotated, Sequence, TypedDict
from langchain_core.messages import HumanMessage, AIMessage
from operator import add

class GraphState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], add]
    #HumanMessage | AIMessage : HumanMessage 객체 or AIMessage 타입 객체를 담을 수 있음
    data: Optional[dict]
    #Optional : None 값 가능
    missing_info: Optional[str]
    summary: Optional[str]
    booking_reference: str

In [16]:
# Node to fetch data from MongoDB
def fetch_data_node(state):
    booking_reference = state['booking_reference']
    data = fetch_data_from_mongodb(COLLECTION_NAME, {"bookingReference": booking_reference})
    return {"data": data}

# Node to check for missing data
def check_missing_node(state):
    data = state['data']
    missing_info = check_missing(json_util.dumps(data))
    return {"missing_info": missing_info}

# Node to summarize data and missing info
def create_summary_node(state):
    data = state['data']
    missing_info = state['missing_info']
    summary = summarize_data(json_util.dumps(data), missing_info)
    return {"summary": summary}

In [18]:
from langgraph.graph import StateGraph, START, END

# Create the graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("fetch_data", fetch_data_node)
workflow.add_node("check_missing", check_missing_node)
workflow.add_node("create_summary", create_summary_node)

# Add edges
workflow.add_edge(START, 'fetch_data')
workflow.add_edge('fetch_data', 'check_missing')
workflow.add_edge('check_missing', 'create_summary')
workflow.add_edge('create_summary', END)

# Compile the graph
graph = workflow.compile()

In [19]:
graph.invoke({"booking_reference": "CHERRY202409072244"})

{'messages': [],
 'data': [{'_id': ObjectId('66fcfef90e5b23189a5fa165'),
   'bookingReference': 'CHERRY202409072244',
   'voyageDetails': {'vesselName': 'APL TEMASEK', 'voyageNumber': '2024581E'},
   'routeDetails': {'placeOfReceipt': 'NINGBO',
    'portOfLoading': 'NINGBO',
    'portOfDischarge': 'YANTIAN',
    'placeOfDelivery': 'YANTIAN'},
   'paymentDetails': {'freightPaymentTerms': 'COLLECT',
    'freightPayableAt': 'ROTTERDAM, NETHERLANDS'},
   'documentationDetails': {'blType': 'NEGOTIABLE',
    'numberOfOriginalBLs': 3,
    'numberOfCopies': 0},
   'partyDetails': {'shipper': {'name': 'SHIPPER 5029',
     'address': 'NO. 188, SHANXI ROAD, NINGBO, CHINA',
     'telephone': '+86 574 8765 4321',
     'email': 'INFO@CHINAIMPORTEXPORT.COM'},
    'consignee': {'name': 'EUROTECH TRADING BV',
     'address': 'WATERLOO PLAZA, 121-123 WATERLOOPLEIN, 1011 PG AMSTERDAM, NETHERLANDS',
     'taxId': 'NL856321452B01',
     'president': 'MR. JAN VAN DER WIEL',
     'telephone': '+31 20 624 350